In [ ]:
import orekit
if orekit.getVMEnv() is None:
    orekit.initVM()

import os
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
if not os.path.exists('orekit-data.zip'):
    download_orekit_data_curdir()
setup_orekit_curdir()

In [ ]:
from datetime import datetime, timedelta
date = datetime(2021, 4, 2, 17, 4, 29)
duration_s = 3600.0  # seconds, minimum duration starting from date that we want to have in the OEM file

Searching for the best OEM file for the given date.

As the ISS OEM files contain both past (i.e. historical, definitive) and predicted orbit data,

In [ ]:
directory_before_2021 = 'ISS.OEM_J2K_EPH'
directory_from_2021 = 'ISS_OEM'

In [ ]:
import urllib.request
from orekit.pyhelpers import absolutedate_to_datetime, datetime_to_absolutedate
from org.orekit.files.ccsds import OEMParser

date_to_try = date
error = True

while error:
    try:
        filename = f'ISS.OEM_J2K_EPH_{date_to_try:%Y-%m-%d}.txt'

        if date_to_try.year < 2021:
            directory = directory_before_2021
        else:
            directory = directory_from_2021

        # Downloading CCSDS OEM file        
        target_url = f'https://nasa-public-data.s3.amazonaws.com/iss-coords/{date_to_try:%Y-%m-%d}/{directory}/ISS.OEM_J2K_EPH.txt'
        urllib.request.urlretrieve(target_url, filename)
        print(f'Successfully downloaded CCSDS OEM for date {date_to_try:%Y-%m-%d}')

        # Replacing wrong keyword USABLE_START_TIME to USEABLE_START_TIME in OEM files prior to August 2021...
        with open(filename) as f:
            newText=f.read().replace('USABLE', 'USEABLE')
        with open(filename, "w") as f:
            f.write(newText)

        # Parsing the OEM file using Orekit
        oem_file = OEMParser().parse(filename)
        # Reading the first ephemeris block in the CCSDS OEM file. 
        # We assume that the ISS OEMs always contain only one block, which seems to be always the case.
        ephem_first_block = oem_file.getEphemeridesBlocks().get(0)

        # Checking if the OEM file contains data for the desired time range
        if absolutedate_to_datetime(ephem_first_block.getStartTime()) > date:
            print('The current OEM file starts after the desired date, searching another file at an earlier date')
            date_to_try = date_to_try + timedelta(days=-1)
        elif absolutedate_to_datetime(ephem_first_block.getStopTime()) < (date + timedelta(seconds=duration_s)):
            print('Something is wrong, the OEM file (normally 15 days long) stops before the desired time range')      
            break
        else:
            print('The current OEM file contains data at the date we want')
            error = False
        
    except:
        print(f'Date {date_to_try:%Y-%m-%d} has no available data, retrying an earlier date')
        date_to_try = date_to_try + timedelta(days=-1)

Getting the PV coordinates

In [ ]:
from org.orekit.files.general import EphemerisFile
# Must cast to the EphemerisFile.EphemerisSegment interface because of the limitations of the Orekit Python wrapper
bounded_propagator = EphemerisFile.EphemerisSegment.cast_(ephem_first_block).getPropagator()

In [ ]:
from org.orekit.frames import FramesFactory
eme2000 = FramesFactory.getEME2000()

In [ ]:
pv_eme2000 = bounded_propagator.getPVCoordinates(datetime_to_absolutedate(date), eme2000)